<a href="https://colab.research.google.com/github/alexisnpavlik/Job-listing-Query/blob/main/SONIC_Experiment_Alexis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Set Up

In [ ]:
# Prepare the environment
%%capture
from google.colab import drive, auth
drive.mount('/content/drive')
%cd /content/drive/MyDrive/python-library
!pip install .
!sudo apt-get install python-pymysql
!pip install colab-env
!pip install Hashids
import colab_env
%cd /content
import nltk
nltk.download('stopwords')
import gallifrey
import pandas as pd
import numpy as np
import time
from hashids import Hashids

#Set up for Hash calculation
hasher = Hashids("jS5qAlR0kHbAeqeuVowPBnyotKuTvvlGzIzkEtz79XQGB7BJHb", 
                 8, 
                 "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ1234567890")

# Authenticate
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials as GC
gc = gspread.authorize(GC.get_application_default())

# create, and save df
from gspread_dataframe import set_with_dataframe

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=4BaqcoVL64jsw7IslvI90bhvMNpANB&prompt=consent&access_type=offline&code_challenge=U6veDQMAQOih98CFXM7jxzovBM-QXCFNruOh_ApLFY4&code_challenge_method=S256

Enter verification code: 4/1AX4XfWjx654bz0BOx5cTZfY8gwZ7DWIUy_UwJZDSKAKzL0PRFVPLKMTCboY


## Manual Update

#### Query


In [ ]:
!rm -r Job-listing-Query
!git clone https://github.com/alexisnpavlik/Job-listing-Query.git

main_query=open("""/content/Job-listing-Query/main_query.sql""", 'r').read()
organizations_query=open("""/content/Job-listing-Query/organizations_query.sql""", 'r').read()
notifications_query=open("""/content/Job-listing-Query/notifications.sql""", 'r').read()
request_manual_invite_query=open("""/content/Job-listing-Query/request_manual_invite.sql""", 'r').read()

Cloning into 'Job-listing-Query'...
remote: Enumerating objects: 357, done.
remote: Counting objects: 100% (357/357), done.
remote: Compressing objects: 100% (246/246), done.
remote: Total 357 (delta 217), reused 245 (delta 109), pack-reused 0
Receiving objects: 100% (357/357), 55.30 KiB | 6.91 MiB/s, done.
Resolving deltas: 100% (217/217), done.


In [ ]:
start_time=time.time()
main = pd.DataFrame(gallifrey.discovery.execute_query(main_query))
organizations_df = pd.DataFrame(gallifrey.solar.execute_query(organizations_query))
notifications_df = pd.DataFrame(gallifrey.snowplow.execute_query(notifications_query))
request_manual_invite_df = pd.DataFrame(gallifrey.solar.execute_query(request_manual_invite_query))
print('Tiempo de ejecución:',time.time()-start_time,'s')

Tiempo de ejecución: 48.099480867385864 s


In [ ]:
main_df = main.copy(deep=True)
main_df

,ID,Job title,Company_id,location,Type of service,Type of job,Created date,Approved date,Applicant Acquisition Coordinator,Commited date,Status,Completed applications,Completed applications yesterday,Incomplete applications,Mutual matches,Others,Disqualified,Last changes,Closing Date,Language of the post,Hires,Hires yesterday,Opportunity approved yesterday,Sharing token
0,1858934,Frontend Engineer,894147.0,Colombia,prime,full-time,2022-02-09,2022-02-09,None,2022-02-09,open,1.0,1.0,0.0,0.0,0.0,0.0,2022-02-09,2022-03-11,es,NaN,NaN,1.0,y9sXxk1VXfQt
1,1858933,Backend Engineer,894147.0,Colombia,prime,full-time,2022-02-09,2022-02-09,None,2022-02-09,open,0.0,0.0,0.0,0.0,0.0,0.0,2022-02-09,2022-03-11,es,NaN,NaN,1.0,uk7zMd5lAGHR
2,1858930,Key Account Manager,405121.0,"Bogotá, Bogota, Colombia",prime,full-time,2022-02-09,2022-02-09,None,2022-02-09,open,0.0,0.0,1.0,0.0,0.0,0.0,2022-02-10,2022-03-11,en,NaN,NaN,1.0,Eg35bwK6BLpu
3,1858926,Business Intelligence Engineer,405121.0,Colombia,prime,full-time,2022-02-09,2022-02-09,None,2022-02-09,open,0.0,0.0,0.0,0.0,0.0,0.0,2022-02-09,2022-03-11,en,NaN,NaN,1.0,sR2y3SmwbqPi
4,1858923,UX/UI Designer,926848.0,None,self_service,full-time,2022-02-09,2022-02-09,None,2022-02-09,open,1.0,1.0,1.0,0.0,0.0,0.0,2022-02-09,2022-03-11,es,NaN,NaN,1.0,t8GeYBIM0TeC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4930,75319,Desarrollador Web - PHP & JS,730852.0,None,self_service,full-time,2021-01-06,2021-01-14,None,None,closed,139.0,0.0,12.0,4.0,15.0,100.0,2021-10-17,2021-02-04,en,1.0,NaN,NaN,dWofLGCm8V39
4931,75318,Desarrollador Flutter,730852.0,None,self_service,full-time,2021-01-06,2021-01-14,None,None,closed,129.0,0.0,5.0,9.0,6.0,100.0,2021-10-17,2021-04-01,es,1.0,NaN,NaN,tDBcd7OtbOVJ
4932,75274,Desarrollador Magento para soporte a Desarroll...,610062.0,None,self_service,part-time,2021-01-06,2021-01-06,None,None,closed,60.0,0.0,6.0,0.0,21.0,39.0,2021-03-05,2021-01-27,es,1.0,NaN,NaN,D3Ea4bojzkAc
4933,75250,Analista administrativo/a,755408.0,Colombia,self_service,full-time,2021-01-06,2021-01-12,None,None,closed,104.0,0.0,17.0,17.0,1.0,30.0,2021-10-09,2021-01-29,es,1.0,NaN,NaN,hjeA51VwDzs0


In [ ]:
organizations_df.index

RangeIndex(start=0, stop=495267, step=1)

In [ ]:
# Alfa ID Generator
def alfa_id(id):
  return hasher.encode(id)

# URL Generator
def url_generator(id):
  return 'https://torre.co/postings/'+hasher.encode(id)

# Manager URL Generator
def manager_generator(id):
  return 'https://torre.co/postings/'+hasher.encode(id)+'/members-list?sharingToken='

# Transforms columns
main_df.insert(1,"Alfa ID", main_df['ID'].apply(alfa_id),True)
main_df.insert( 2,"URL", main_df['ID'].apply(url_generator),True)

main_df.insert( 6,"Manager link", 'https://torre.co/postings/'+ main_df['Alfa ID'] +'/members-list?sharingToken='+ main_df['Sharing token'] ,True)

In [ ]:
notifications_df.dtypes

alfaid          object
realistic        int64
people           int64
organization     int64
dtype: object

In [ ]:
notifications_df = notifications_df.rename(columns={'alfaid':'Alfa ID','realistic':'Sent Matches','people':'Sent SigPeop','organization': 'Sent SigOrg'})

request_manual_invite_df = request_manual_invite_df.rename(columns={'AlfaID':'Alfa ID','sent':'Sent manual invite','pending':'Pending manual invite'})

In [ ]:
opportunities_df = pd.merge(left=main_df,right=organizations_df)
opportunities_df = pd.merge(left=opportunities_df,right=notifications_df)
opportunities_df = pd.merge(left=opportunities_df,right=request_manual_invite_df)

opportunities_df = opportunities_df.reindex(columns=['ID','Job title','Company','Alfa ID','location','Created date','Approved date','Commited date','Applicant Acquisition Coordinator','Type of job',
                                             'Type of service','URL','Manager link','Language of the post','Status','Last changes','Closing Date',
                                             'Completed applications','Completed applications yesterday','Incomplete applications','Mutual matches','Opportunity approved yesterday','Disqualified','Others','Hires','Hires yesterday',
                                             'Sent Matches','Sent SigPeop','Sent SigOrg','Sent manual invite','Pending manual invite'])

In [ ]:
sheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1V1RkMTrOFEQLsYGJaA9KukO9fgtgf6uwG8o-QHrkk2Q/edit#gid=0').sheet1

sheet.clear()

set_with_dataframe(sheet, opportunities_df) 

# Update Google Sheet

In [ ]:
sheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1V1RkMTrOFEQLsYGJaA9KukO9fgtgf6uwG8o-QHrkk2Q/edit#gid=0').sheet1

sheet.clear()

set_with_dataframe(sheet, opportunities_df) 

# Save to Google drive

##### Save csv

In [ ]:
drive.mount('/content/drive')

In [ ]:
opportunities_df.to_csv('/content/drive/My Drive/Data_studio_dataset/Dataset.csv',index=False,encoding='utf-8')

##### Save  Excel

In [ ]:
drive.mount('/content/drive')

In [ ]:
opportunities_df.to_excel('/drive/My Drive/Data_studio_dataset/Dataset.xls',index=False,encoding='utf-8')

# Testing

In [ ]:
query_testing = """

select id as 'Company_id', name as 'Company' from organizations ;

"""

In [ ]:
Testing = pd.DataFrame(gallifrey.solar.execute_query(query_testing))

In [ ]:
!rm -r Job-listing-Query
!git clone https://github.com/alexisnpavlik/Job-listing-Query.git

main_query=open("""/content/Job-listing-Query/testing.sql""", 'r').read()
organizations_query=open("""/content/Job-listing-Query/organizations_query.sql""", 'r').read()

Cloning into 'Job-listing-Query'...
remote: Enumerating objects: 274, done.
remote: Counting objects: 100% (274/274), done.
remote: Compressing objects: 100% (190/190), done.
remote: Total 274 (delta 166), reused 186 (delta 82), pack-reused 0
Receiving objects: 100% (274/274), 44.00 KiB | 464.00 KiB/s, done.
Resolving deltas: 100% (166/166), done.


### Update Google Sheet

In [ ]:
sheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1V1RkMTrOFEQLsYGJaA9KukO9fgtgf6uwG8o-QHrkk2Q/edit#gid=0').sheet1

sheet.clear()

set_with_dataframe(sheet, Testing) 

In [ ]:
Testing.head()

,Company_id,Company
0,795908,
1,773085,0 2 Convergencia Digital S A S
2,763230,2 Way Sports
3,795148,23Ky Colombia S.A.S.
4,891553,60 Decibels
